<a href="https://colab.research.google.com/github/rizkiar00/NaiveBayes/blob/master/NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IMPORT

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt; 
from google.colab import files
from sklearn.model_selection import train_test_split
import math
import operator 
%matplotlib inline
!git clone https://github.com/rizkiar00/NaiveBayes

fatal: destination path 'NaiveBayes' already exists and is not an empty directory.


## PISAH TRAINING DAN TESTING

In [99]:
data = pd.read_csv('NaiveBayes/TrainsetTugas1ML.csv',skipinitialspace=True) #Masukkan Data dan Buat Data Training,Testing
datasmall = data.loc[data['income'] == '>50K'].reset_index()
databig = data.loc[data['income'] == '<=50K'].reset_index()
datatrain = pd.concat([datasmall.loc[0:85],databig.loc[0:35]], ignore_index=True).drop(["index"], axis=1)
datatest = pd.concat([datasmall.loc[85:],databig.loc[35:]], ignore_index=True).drop(["index"], axis=1)
datatrainbig = datatrain.loc[datatrain['income'] == '>50K'].reset_index().drop(["index"], axis=1)
datatrainsmall = datatrain.loc[datatrain['income'] == '<=50K'].reset_index().drop(["index"], axis=1)
datatrainsmall

,id,age,workclass,education,marital-status,occupation,relationship,hours-per-week,income
0,1818,adult,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,normal,<=50K
1,9606,adult,Self-emp-not-inc,Some-college,Divorced,Craft-repair,Not-in-family,normal,<=50K
2,8930,adult,Private,HS-grad,Divorced,Craft-repair,Not-in-family,normal,<=50K
3,24980,young,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,normal,<=50K
4,12500,adult,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,normal,<=50K
5,30039,young,Private,HS-grad,Never-married,Craft-repair,Own-child,normal,<=50K
6,23050,adult,Self-emp-not-inc,Bachelors,Never-married,Exec-managerial,Own-child,normal,<=50K
7,27672,adult,Private,Bachelors,Never-married,Craft-repair,Not-in-family,normal,<=50K
8,21118,young,Private,HS-grad,Never-married,Craft-repair,Own-child,low,<=50K
9,25440,adult,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,normal,<=50K


## HITUNG PRIOR

In [100]:
totalbig = datatrainbig['workclass'].count()+1 #Tambah satu buat di laplace smoothing
totalsmall = datatrainsmall['workclass'].count()+1

youngbig = (datatrainbig['age'][datatrainbig['age'] == 'young'].count()+1)
adultbig = (datatrainbig['age'][datatrainbig['age'] == 'adult'].count()+1)
oldbig = (datatrainbig['age'][datatrainbig['age'] == 'old'].count()+1)
youngsmall = (datatrainsmall['age'][datatrainsmall['age'] == 'young'].count()+1)
adultsmall = (datatrainsmall['age'][datatrainsmall['age'] == 'adult'].count()+1)
oldsmall = (datatrainsmall['age'][datatrainsmall['age'] == 'old'].count()+1)
d = {'': ["young", "adult","old"], 'big': [youngbig, adultbig,oldbig], 'small': [youngsmall , adultsmall, oldsmall]}
umur = pd.DataFrame(data=d)

Privatebig = (datatrainbig['workclass'][datatrainbig['workclass'] == 'Private'].count()+1)
Selfbig = (datatrainbig['workclass'][datatrainbig['workclass'] == 'Self-emp-not-inc'].count()+1)
Localbig = (datatrainbig['workclass'][datatrainbig['workclass'] == 'Local-gov'].count()+1)
Privatesmall = (datatrainsmall['workclass'][datatrainsmall['workclass'] == 'Private'].count()+1)
Selfsmall = (datatrainsmall['workclass'][datatrainsmall['workclass'] == 'Self-emp-not-inc'].count()+1)
Localsmall = (datatrainsmall['workclass'][datatrainsmall['workclass'] == 'Local-gov'].count()+1)
d = {'': ["Private", "Self-emp-not-inc","Local-gov"], 'big': [Privatebig, Selfbig,Localbig], 'small': [Privatesmall , Selfsmall, Localsmall]}
workclass = pd.DataFrame(data=d)

Somebig = (datatrainbig['education'][datatrainbig['education'] == 'Some-college'].count()+1)
Bachelorsbig = (datatrainbig['education'][datatrainbig['education'] == 'Bachelors'].count()+1)
HSbig = (datatrainbig['education'][datatrainbig['education'] == 'HS-grad'].count()+1)
Somesmall = (datatrainsmall['education'][datatrainsmall['education'] == 'Some-college'].count()+1)
Bachelorssmall = (datatrainsmall['education'][datatrainsmall['education'] == 'Bachelors'].count()+1)
HSsmall = (datatrainsmall['education'][datatrainsmall['education'] == 'HS-grad'].count()+1)
d = {'': ["Some-college", "Bachelors","HS-grad"], 'big': [Somebig, Bachelorsbig,HSbig], 'small': [Somesmall , Bachelorssmall, HSsmall]}
education = pd.DataFrame(data=d)

Husbandbig = (datatrainbig['relationship'][datatrainbig['relationship'] == 'Husband'].count()+1)
Notbig = (datatrainbig['relationship'][datatrainbig['relationship'] == 'Not-in-family'].count()+1)
Ownbig = (datatrainbig['relationship'][datatrainbig['relationship'] == 'Own-child'].count()+1)
Husbandsmall = (datatrainsmall['relationship'][datatrainsmall['relationship'] == 'Husband'].count()+1)
Notsmall = (datatrainsmall['relationship'][datatrainsmall['relationship'] == 'Not-in-family'].count()+1)
Ownsmall = (datatrainsmall['relationship'][datatrainsmall['relationship'] == 'Own-child'].count()+1)
d = {'': ["Husband", "Not-in-family","Own-child"], 'big': [Husbandbig, Notbig,Ownbig], 'small': [Husbandsmall , Notsmall, Ownsmall]}
relationship = pd.DataFrame(data=d)

normalbig = (datatrainbig['hours-per-week'][datatrainbig['hours-per-week'] == 'normal'].count()+1)
lowbig = (datatrainbig['hours-per-week'][datatrainbig['hours-per-week'] == 'low'].count()+1)
manybig = (datatrainbig['hours-per-week'][datatrainbig['hours-per-week'] == 'many'].count()+1)
normalsmall = (datatrainsmall['hours-per-week'][datatrainsmall['hours-per-week'] == 'normal'].count()+1)
lowsmall = (datatrainsmall['hours-per-week'][datatrainsmall['hours-per-week'] == 'low'].count()+1)
manysmall = (datatrainsmall['hours-per-week'][datatrainsmall['hours-per-week'] == 'many'].count()+1)
d = {'': ["normal", "low","many"], 'big': [normalbig, lowbig,manybig], 'small': [normalsmall , lowsmall,manysmall]}
hours = pd.DataFrame(data=d)

# normal = datatrain['hours-per-week'][data['hours-per-week'] == 'normal'].count()/total
# low = datatrain['hours-per-week'][data['hours-per-week'] == 'low'].count()/total

# lebih = datatrain['income'][data['income'] == '<=50K'].count()
# kurang = datatrain['income'][data['income'] == '>50K'].count()

# data_means = np.var([lebih,kurang])

print(umur)
print() 
print(workclass)
print() 
print(education)
print() 
print(relationship)
print() 
print(hours) 
print() 

          big  small
0  young   48     21
1  adult   40     17
2    old    1      1

                     big  small
0           Private   75     30
1  Self-emp-not-inc    7      8
2         Local-gov    7      1

                 big  small
0  Some-college   20     14
1     Bachelors   47      8
2       HS-grad   22     17

                  big  small
0        Husband   74     18
1  Not-in-family   12     13
2      Own-child    3      8

           big  small
0  normal   86     31
1     low    2      5
2    many    1      3

